In [1]:
import pickle

In [7]:
file = open("smaller_station_data.pickle","rb")
station_weather = pickle.load(file)
file.close()

In [8]:

len(station_weather.keys())

1218

In [97]:
def get_values(filename):
    
    maxT = 0
    minT = 0
    avgP = 0
    max_counter = 1
    min_counter = 1
    precip_counter = 1
    #year = []
    for i in range(0,len(station_weather[filename])):
        element = station_weather[filename].iloc[i].ELEMENT
        #if station_weather[filename].iloc[i].YEAR not in year:
            #year.append(station_weather[filename].iloc[i].YEAR)
        if element == "TMAX":
            for j in range(1,32):
                value="VALUE"+str(j)
                if station_weather[filename].iloc[i][value] == -9999:
                    max_counter +=1
                elif station_weather[filename].iloc[i][value] > maxT:
                    maxT = station_weather[filename].iloc[i][value]
        elif element == "TMIN":
            for j in range(1,32):
                value="VALUE"+str(j)
                if station_weather[filename].iloc[i][value] == -9999:
                    min_counter +=1
                elif station_weather[filename].iloc[i][value] < minT:
                    minT = station_weather[filename].iloc[i][value]
        elif element == "PRCP":
            for j in range(1,32):
                value="VALUE"+str(j)
                if station_weather[filename].iloc[i][value] == -9999:
                    precip_counter +=1
                else:
                    avgP += station_weather[filename].iloc[i][value]
    
        avgP = avgP/(31) #tenths of mm

    maxT = maxT/10 #tenths of deg C
    minT = minT/10 #tenths of deg C
    return maxT, minT, avgP

In [48]:
import pandas as pd

In [109]:
df_station = station_info

In [110]:
df_station = df_station.drop(columns=["ELEV","ST","NAME","?","GSN","HCN","WMOID"])

In [111]:
df_station["LAT"] = df_station["LAT"].apply(lambda x: round(x,1))
df_station["LON"] = df_station["LON"].apply(lambda x: round(x,1))

In [112]:
maxT_df = df_station.drop(columns="LON")
minT_df = df_station.drop(columns="LON")
precip_df = df_station.drop(columns="LON")
maxT_df = maxT_df.rename(columns={"LAT":"maxT"})
minT_df = minT_df.rename(columns={"LAT":"minT"})
precip_df = precip_df.rename(columns={"LAT":"precip"})

In [114]:
maxT_list = []
minT_list = []
precip_list = []

In [115]:

for i in range(0,len(df_station)):
    station = df_station.iloc[i].ID
    filename = "./ghcnd_hcn/ghcnd_hcn/"+station+".dly"
    values = get_values(filename)
    maxT_list.append(values[0])
    minT_list.append(values[1])
    precip_list.append(values[2])

In [116]:

with open('maxT.pickle', 'wb') as f:
    pickle.dump(maxT_list, f)
with open('minT.pickle', 'wb') as f:
    pickle.dump(minT_list, f)
with open('precip.pickle', 'wb') as f:
    pickle.dump(precip_list, f)    

In [120]:
maxT_df["maxT"]=maxT_list
minT_df["minT"]=minT_list
precip_df["precip"]=precip_list


In [122]:
minT_df.head

<bound method NDFrame.head of                  ID  minT
0       USW00094008 -35.6
92086   USC00011084  -9.4
92147   USC00012813  -8.3
92162   USC00013160 -12.2
92172   USC00013511 -11.1
...             ...   ...
118603  USW00094224  -5.0
118635  USW00094728 -17.2
118645  USW00094793   0.0
118646  USW00094794 -29.4
118691  USW00094967 -42.8

[1218 rows x 2 columns]>

In [123]:
df_station = pd.merge(df_station,maxT_df)
df_station = pd.merge(df_station,minT_df)
df_station = pd.merge(df_station,precip_df)

In [124]:
df_station.head

<bound method NDFrame.head of                ID   LAT    LON  maxT  minT     precip
0     USW00094008  48.2 -106.6  40.6 -35.6   0.003256
1     USC00011084  31.1  -87.1  38.9  -9.4   0.030446
2     USC00012813  30.5  -87.9  38.9  -8.3  32.033145
3     USC00013160  32.8  -88.1  39.4 -12.2  32.032769
4     USC00013511  32.7  -87.6  40.0 -11.1  15.322608
...           ...   ...    ...   ...   ...        ...
1213  USW00094224  46.2 -123.9  35.6  -5.0   0.122420
1214  USW00094728  40.8  -74.0  39.4 -17.2   0.039206
1215  USW00094793  41.2  -71.6   0.0   0.0   0.000000
1216  USW00094794  43.1  -75.4  36.1 -29.4   0.016112
1217  USW00094967  46.9  -95.1  36.1 -42.8   0.003122

[1218 rows x 6 columns]>

In [125]:
with open('combined.pickle', 'wb') as f:
    pickle.dump(df_station, f)  

In [ ]:
station_weather["./ghcnd_hcn/ghcnd_hcn/USC00404561.dly"]

In [13]:
file = open("hcn_stations.pickle","rb")
station_info = pickle.load(file)
file.close()

In [131]:

df_station.to_csv("combined.csv")

In [126]:
import pyspark

In [143]:

from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, StructField, StructType, DoubleType, IntegerType

In [128]:
spark = (pyspark.sql.SparkSession.builder.getOrCreate())

In [145]:
schema = StructType([StructField("_c0",IntegerType(),True),
                    StructField("ID", StringType(), True),
                     StructField("LAT", DoubleType(), True),
                     StructField("LON", DoubleType(), True),
                     StructField("maxT", DoubleType(), True),
                     StructField("minT", DoubleType(), True),
                     StructField("precip", DoubleType(), True)])

In [146]:
combined_df = spark.read.csv("combined.csv",header="true",schema=schema)

In [147]:
combined_df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- ID: string (nullable = true)
 |-- LAT: double (nullable = true)
 |-- LON: double (nullable = true)
 |-- maxT: double (nullable = true)
 |-- minT: double (nullable = true)
 |-- precip: double (nullable = true)



In [148]:
combined_df.show(5)

+---+-----------+----+------+----+-----+--------------------+
|_c0|         ID| LAT|   LON|maxT| minT|              precip|
+---+-----------+----+------+----+-----+--------------------+
|  0|USW00094008|48.2|-106.6|40.6|-35.6|0.003256017105458339|
|  1|USC00011084|31.1| -87.1|38.9| -9.4|0.030446342453921964|
|  2|USC00012813|30.5| -87.9|38.9| -8.3|   32.03314489020785|
|  3|USC00013160|32.8| -88.1|39.4|-12.2|   32.03276915252046|
|  4|USC00013511|32.7| -87.6|40.0|-11.1|  15.322607766248499|
+---+-----------+----+------+----+-----+--------------------+
only showing top 5 rows

